<a href="https://colab.research.google.com/github/nabilfzn/Sentiment-analysis/blob/main/review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/folder dataframe/reviews.csv')
df.head()

,date,review,rating,thumbs_up,version
0,2021-09-30 06:12:53,"Udah di coba, keren dan responsive, dengan tam...",5,36,1.0.0
1,2021-09-30 06:33:15,Excellent,5,0,1.0.0
2,2021-09-30 06:48:30,Keren. Cakep benar semakin canggih. Terdepan t...,5,22,1.0.0
3,2021-09-30 06:56:05,mantap,5,0,NaN
4,2021-09-30 07:02:21,Mantap,5,0,1.0.0


In [ ]:
df = df.sample(n=1000, random_state=42)

In [ ]:
df.rating.value_counts()

,count
rating,
5,563
1,238
2,76
4,62
3,61


In [ ]:
# mengelompokkan kelas review

df.rating = df.rating.replace({
    1: 'negatif',
    2: 'negatif',
    3: 'netral',
    4: 'positif',
    5: 'positif'
})

df[df.rating == 'negatif'].sample(10)

,date,review,rating,thumbs_up,version
50056,2022-02-25 08:58:16,"Muter2 terus, ga bisa transfer, jaringan data ...",negatif,0,1.0.2
138462,2022-11-02 02:59:49,Aplikasi tidak bisa di gunakan GK bisa di klik...,negatif,0,NaN
19297,2021-12-24 18:16:29,"Untuk Pembayaran e commerce, SpayLetter dan Sh...",negatif,0,NaN
150816,2022-12-12 10:47:15,Maaf tolong ini gmana yaa ko saya ga bisa top ...,negatif,1,1.1.3
66183,2022-04-26 02:58:33,Mau bayar atau mau transfer kenapa tidak bisa ...,negatif,0,1.0.2
15686,2021-12-04 08:40:54,"Jelek,bolak balik di suruh verifikasi tapi cob...",negatif,0,NaN
84069,2022-06-06 14:45:51,Kok baru di update makin gk bisa di login kan ...,negatif,0,1.1.0
51971,2022-02-25 11:48:34,"Tolong dong di perhatiin, masa saya mau masuk ...",negatif,0,1.0.2
42960,2022-02-04 18:42:39,"Tidak bisa load akun rekening, suruh coba muat...",negatif,0,1.0.2
50983,2022-02-25 10:58:34,Kecewa dengan apk ini... mau masuk gagal login...,negatif,0,1.0.2


In [ ]:
# duplicate
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [ ]:
# data null
df.isnull().sum()

,0
date,0
review,0
rating,0
thumbs_up,0
version,233


In [ ]:
#drop kolom tidak berguna
df.drop(columns=['thumbs_up', 'version', 'date'], inplace=True)

In [ ]:
df

,review,rating
110537,Luar biasa si,positif
43991,"aplikasi mudah digunakan, nggak ribet buat pemula",positif
57870,Gabisa masuk,negatif
36724,"Unggah di tanda tangan susah sekali, gk bisa t...",negatif
81956,Sangat membantu dan tidak ribet untuk bertrans...,positif
...,...,...
141772,"Bisa lebih cepat bertransaksi, terimakasih.",positif
145036,Bentar bentar update,positif
1077,Verifikasi ga jadi2..gagal trus,negatif
129792,Biaya transfer beda bank cukup murah,positif


# PREPROCESSING

In [ ]:
# case folding

import re

def casefolding(komen):
  komen = re.sub(r'\d+', '', komen)
  komen = komen.lower()
  komen = komen.strip(' ')
  komen = re.sub(r'[!"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~]', '', komen)
  komen = re.sub(r',\s*,', ', ', komen)  # Membersihkan koma berganda
  komen = re.sub(r'\s+', ' ', komen)  # Menghapus spasi berlebih
  komen = komen.strip()  # Menghapus spasi di awal/akhir teks

  return komen

df['review'] = df['review'].apply(casefolding)
df

,review,rating
110537,luar biasa si,positif
43991,aplikasi mudah digunakan nggak ribet buat pemula,positif
57870,gabisa masuk,negatif
36724,unggah di tanda tangan susah sekali gk bisa te...,negatif
81956,sangat membantu dan tidak ribet untuk bertrans...,positif
...,...,...
141772,bisa lebih cepat bertransaksi terimakasih,positif
145036,bentar bentar update,positif
1077,verifikasi ga jadigagal trus,negatif
129792,biaya transfer beda bank cukup murah,positif


In [ ]:
#remove emoji

def emoji_remove (komen):
  emoji_pattern = re.compile(
       "["
        u"\U0001F600-\U0001F64F"  # Emoticon
        u"\U0001F300-\U0001F5FF"  # Simbol & Objek
        u"\U0001F680-\U0001F6FF"  # Transportasi & Simbol
        u"\U0001F1E0-\U0001F1FF"  # Bendera
        u"\U00002702-\U000027B0"  # Simbol lain
        u"\U000024C2-\U0001F251"  # Simbol tambahan
        u"\U0001F910-\U0001F9FF"  # Emoji tambahan
        "]+",
        flags=re.UNICODE
  )
  komen = emoji_pattern.sub(r'', komen)
  return komen

df['review'] = df['review'].apply(emoji_remove)
df.head(10)

,review,rating
110537,luar biasa si,positif
43991,aplikasi mudah digunakan nggak ribet buat pemula,positif
57870,gabisa masuk,negatif
36724,unggah di tanda tangan susah sekali gk bisa te...,negatif
81956,sangat membantu dan tidak ribet untuk bertrans...,positif
72507,sangat bagus,positif
44734,topup ewallet lebih mudah dan cepat good aplikasi,positif
75510,jaringan kadang masih error,positif
87378,sangat membantu,positif
97529,josss,positif


In [ ]:
# tokenizing

def tokenize(komen):
  komen = komen.split()
  return komen

df['review'] = df['review'].apply(tokenize)
df.head(10)

,review,rating
110537,"[luar, biasa, si]",positif
43991,"[aplikasi, mudah, digunakan, nggak, ribet, bua...",positif
57870,"[gabisa, masuk]",negatif
36724,"[unggah, di, tanda, tangan, susah, sekali, gk,...",negatif
81956,"[sangat, membantu, dan, tidak, ribet, untuk, b...",positif
72507,"[sangat, bagus]",positif
44734,"[topup, ewallet, lebih, mudah, dan, cepat, goo...",positif
75510,"[jaringan, kadang, masih, error]",positif
87378,"[sangat, membantu]",positif
97529,[josss],positif


In [ ]:
!pip install sastrawi

In [ ]:
#stop word removal

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopwords = factory.create_stop_word_remover()

def stopword_remove(komen):
  return stopwords.remove(' '.join(komen)).split()

df['review'] = df['review'].apply(stopword_remove)
df.head(10)

,review,rating
110537,"[luar, biasa, si]",positif
43991,"[aplikasi, mudah, digunakan, ribet, buat, pemula]",positif
57870,"[gabisa, masuk]",negatif
36724,"[unggah, tanda, tangan, susah, sekali, gk, ter...",negatif
81956,"[sangat, membantu, tidak, ribet, bertransaksi,...",positif
72507,"[sangat, bagus]",positif
44734,"[topup, ewallet, lebih, mudah, cepat, good, ap...",positif
75510,"[jaringan, kadang, error]",positif
87378,"[sangat, membantu]",positif
97529,[josss],positif


In [ ]:
df.shape

(1000, 2)

In [ ]:
# stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(komen):
  factory = StemmerFactory()
  stem = factory.create_stemmer()
  komen = [stem.stem(word) for word in komen]
  return komen

df['review'] = df['review'].apply(stemming)
df.to_csv('df_clean.csv')

In [ ]:
#tfidf

#Proses TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1,2))
text_tf = tf.fit_transform(df['review'].astype('U'))
text_tf

<1000x6514 sparse matrix of type '<class 'numpy.float64'>'
	with 12311 stored elements in Compressed Sparse Row format>

In [ ]:
from imblearn.over_sampling import SMOTE
import pandas as pd

# Misalkan Anda memiliki data dalam DataFrame 'data' dan telah melakukan proses TF-IDF sebelumnya

# Membuat DataFrame baru yang berisi fitur (X) dan label (y)
X_tfidf = text_tf  # Hasil dari proses TF-IDF
y = df['rating']  # Label

# Membuat instance SMOTE
smote = SMOTE()

# Melakukan oversampling pada data yang sudah diubah ke representasi numerik (TF-IDF)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Sekarang Anda dapat menggunakan X_resampled dan y_resampled untuk melatih model Anda

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

x_train, x_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=0)

In [ ]:
#Modelling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

knn = KNeighborsClassifier(n_neighbors=1).fit(x_train, y_train)
pred = knn.predict(x_test)

print("Accuracy : ", accuracy_score(y_test, pred))
print("Precision : ", precision_score(y_test, pred, average='micro'))
print("Recall : ", recall_score(y_test, pred, average='micro'))
print("F1 Score : ", f1_score(y_test, pred, average='micro'))

print(f'Confusion Matrix : \n {confusion_matrix(y_test, pred)}')

Accuracy :  0.8693333333333333
Precision :  0.8693333333333333
Recall :  0.8693333333333333
F1 Score :  0.8693333333333333
Confusion Matrix : 
 [[117   1   8]
 [  0 124   1]
 [ 36   3  85]]


In [ ]:
knn.score(x_test, y_test)

0.8693333333333333

In [ ]:
# logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)
predlr = lr.predict(x_test)

In [ ]:
print("Accuracy : ", accuracy_score(y_test, predlr))
print("Precision : ", precision_score(y_test, predlr, average='micro'))
print("Recall : ", recall_score(y_test, predlr, average='micro'))
print("F1 Score : ", f1_score(y_test, predlr, average='micro'))

print(f'Confusion Matrix : \n {confusion_matrix(y_test, predlr)}')

Accuracy :  0.9333333333333333
Precision :  0.9333333333333333
Recall :  0.9333333333333333
F1 Score :  0.9333333333333333
Confusion Matrix : 
 [[118   0   8]
 [  0 121   4]
 [  8   5 111]]
